In [160]:
import pandas as pd
import numpy as np
import sqlite3
%matplotlib inline

In [161]:
# read in the data
qcourse_df = pd.read_csv('qcourses.csv')
course_df = pd.read_csv('courses.csv', escapechar="\\")
dept_df = pd.read_csv('departments.csv')

In [162]:
# convert workload and overall ratings to numeric
qcourse_df = (qcourse_df.drop(['Workload', 'CourseOverall'], axis=1)
              .join(qcourse_df[['Workload', 'CourseOverall']].apply(pd.to_numeric, errors='coerce')))

In [166]:
pd.to_numeric(qcourse_df['ResponseRate'], errors='coerce').dropna().mean()

79.41169244409417

In [131]:
# drop courses with no values for Workload or CourseOverall
qcourse_df = qcourse_df[~pd.isnull(qcourse_df['Workload'])]
qcourse_df = qcourse_df[~pd.isnull(qcourse_df['CourseOverall'])]
# drop courses without catalogue numbers
qcourse_df = qcourse_df[~pd.isnull(qcourse_df['cat_num'])]
course_df = course_df[~pd.isnull(course_df['cat_num'])]

In [132]:
qcourse_df['cat_num'] = qcourse_df['cat_num'].astype(int)
course_df['cat_num'] = course_df['cat_num'].astype(int)

In [133]:
dept_df['department_id'] = dept_df['id']
dept_df['dept_name'] = dept_df['short_name']
dept_df['department_id'] = dept_df['department_id'].astype(int)

In [134]:
# get department and course names
df = qcourse_df[['course_id', 'Workload', 'CourseOverall', 'Evaluations', 'cat_num', 'year']]\
        .merge(course_df[['cat_num', 'title', 'department_id', 'fall']], on='cat_num', how='left')\
        .merge(dept_df[['department_id', 'dept_name']], on='department_id', how='left')

In [135]:
# filter from fall 2014 onward
df = df[(df['year'] > 2014) | ((df['year'] == 2014) & (df['fall'] == 'Y'))]

In [159]:
df[(df['year'] == 2016) & (df['fall'] == 'Y')]

,course_id,Workload,CourseOverall,Evaluations,cat_num,year,title,department_id,fall,dept_name
18889,80,2.6,4.6,58,123435,2016,"Poverty, Race, and Health",1,Y,African and African American Studies
18895,6,5.0,5.0,2,118965,2016,African Language Tutorials,1,Y,African and African American Studies
18896,7,3.2,4.8,13,126300,2016,Elementary Amharic,525,Y,AMHARIC
18897,10628,2.0,5.0,10,109427,2016,Elementary Afrikaans,517,Y,AFRIKAAN
18898,11510,2.3,4.0,3,110410,2016,Malagasy,1,Y,African and African American Studies
18899,12857,2.9,4.7,14,156750,2016,Elementary Jamaican Patois,526,Y,JAMAICAN
18900,14699,3.5,4.5,2,204110,2016,Elementary West African Pidgin,527,Y,WSTAFRCN
18901,12,4.7,4.5,6,126306,2016,Elementary Haitian Creole,520,Y,HAITIAN
18902,14,3.2,3.9,8,126308,2016,Elementary Igbo,522,Y,IGBO
18903,15,3.0,5.0,1,126309,2016,Kinyarwanda,1,Y,African and African American Studies


In [136]:
df.shape

(4765, 10)

In [137]:
df['CourseOverall'].corr(df['Workload'])

0.011770822214090787

In [140]:
dept_count_df = df.groupby('dept_name')['CourseOverall'].count().reset_index().sort_values(by='CourseOverall', ascending=False)

In [143]:
most_popular_departments = dept_count_df.iloc[:10,:]['dept_name']

In [148]:
pop_dept_corrs = df[df['dept_name'].isin(most_popular_departments)].groupby('dept_name')[['CourseOverall', 'Workload']].corr().ix[0::2,'Workload'].reset_index()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [149]:
pop_dept_counts = df[df['dept_name'].isin(most_popular_departments)].groupby('dept_name').count()['course_id'].reset_index()

In [153]:
pop_dept_df = pop_dept_corrs.merge(pop_dept_counts, on='dept_name')[['dept_name', 'Workload', 'course_id']]

In [157]:
pop_dept_df.rename({'dept_name': 'Department Name', 'Workload': 'Correlation Value', 'course_id': 'Sample Size'},
                   axis='columns').to_csv('correlation_by_department.csv', index=False)